In [8]:
import pandas as pd
import os
import django
import sys
import nest_asyncio
import asyncio
from asgiref.sync import sync_to_async

# Aplicar nest_asyncio para permitir la ejecución de bucles de eventos anidados
nest_asyncio.apply()

# Asegúrate de que la raíz del proyecto esté en el path de Python
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("insert_into_mysql.ipynb"))))
# Configura la variable de entorno DJANGO_SETTINGS_MODULE
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'storefront.settings')

# Configura Django
django.setup()

from store.models import Supplier, Category



# 2. Import data

In [2]:
# Melted version of plata and acero files
df = pd.read_excel("data/clean_files/all_plata_melted.xlsx")
df.head()

,fecha_compra,codigo,talla,peso,costo_gramo,costo,pvp,detalle,proveedor,nota,source,fecha_ingreso,count_items
0,2021-11-22,AP001,T6,2.3,2.4,5.52,18.7,llano 2 máscaras,VOGA,NaN,anillos_plata,2023-03-14,2
1,2021-11-22,AP001,T7,2.3,2.4,5.52,18.7,llano 2 máscaras,VOGA,NaN,anillos_plata,2023-03-14,1
2,2021-11-22,AP001,T9,2.3,2.4,5.52,18.7,llano 2 máscaras,VOGA,NaN,anillos_plata,2023-03-14,1
3,2021-11-22,AP002,T4,1.4,2.4,3.36,17.8,llano circulo figura grabada,VOGA,NaN,anillos_plata,2023-03-14,2
4,2021-11-22,AP002,T5,1.4,2.4,3.36,17.8,llano circulo figura grabada,VOGA,NaN,anillos_plata,2023-03-14,1


#

## 2.1 Proveedores/Suppliers
```
class Supplier(models.Model):
    name = models.CharField(max_length=100)
    contact_info = models.TextField()
    created_at = models.DateTimeField(auto_now_add=True)
    updated_at = models.DateTimeField(auto_now=True)

    def __str__(self):
        return self.name```

# TODO: Split contact info into Address, Phone, RUC

In [3]:
async def insert_suppliers(suppliers):
    for index, row in suppliers.iterrows():
        await sync_to_async(Supplier.objects.create)(
            name=row['name'],
            contact_info=row['contact_info'],
        )

In [4]:
suppliers = pd.DataFrame({
    "name" : df.proveedor.value_counts().index,
    "contact_info" : "MISSING"
    }
)

# Ejecutar la corrutina en el bucle de eventos actual
loop = asyncio.get_event_loop()
loop.run_until_complete(insert_suppliers(suppliers))

## 2.2 Categorías

```
# Tabla de Categorías
class Category(models.Model):
    name = models.CharField(max_length=100)
    description = models.TextField()
    created_at = models.DateTimeField(auto_now_add=True)
    updated_at = models.DateTimeField(auto_now=True)

    def __str__(self):
        return self.name

```

In [10]:
categories = pd.DataFrame({
    "name" :["plata","acero"],
    "description" : ["Joyas de plata", "Joyas de acero"]
    }
)

async def insert_categories(categories):
    for index, row in categories.iterrows():
        await sync_to_async(Category.objects.create)(
            name=row['name'],
            description=row['description'],
        )

# Ejecutar la corrutina en el bucle de eventos actual
loop = asyncio.get_event_loop()
loop.run_until_complete(insert_categories(categories))

## 2.3 Producto

```
class Product(models.Model):
    name = models.CharField(max_length=100)
    description = models.TextField()
    category = models.ForeignKey(Category, on_delete=models.CASCADE)
    price = models.DecimalField(max_digits=10, decimal_places=2)
    cost = models.DecimalField(max_digits=10, decimal_places=2, default=0)
    created_at = models.DateTimeField(auto_now_add=True)
    updated_at = models.DateTimeField(auto_now=True)

    def update_cost(self):
        inventory_entries = Inventory.objects.filter(product=self)
        total_cost = inventory_entries.aggregate(total_cost=models.Sum(models.F('cost') * models.F('quantity')))['total_cost']
        total_quantity = inventory_entries.aggregate(total_quantity=models.Sum('quantity'))['total_quantity']

        if total_quantity and total_quantity > 0:
            average_cost = total_cost / total_quantity
            self.cost = average_cost
            self.save()

    def __str__(self):
        return self.name

    def get_stock(self):
        total_stock = Inventory.objects.filter(product=self).aggregate(total=models.Sum('quantity'))['total']
        return total_stock if total_stock else 0
```

TODO: Añadir peso, talla, costo_gramo

In [12]:
df.head()

,fecha_compra,codigo,talla,peso,costo_gramo,costo,pvp,detalle,proveedor,nota,source,fecha_ingreso,count_items
0,2021-11-22,AP001,T6,2.3,2.4,5.52,18.7,llano 2 máscaras,VOGA,NaN,anillos_plata,2023-03-14,2
1,2021-11-22,AP001,T7,2.3,2.4,5.52,18.7,llano 2 máscaras,VOGA,NaN,anillos_plata,2023-03-14,1
2,2021-11-22,AP001,T9,2.3,2.4,5.52,18.7,llano 2 máscaras,VOGA,NaN,anillos_plata,2023-03-14,1
3,2021-11-22,AP002,T4,1.4,2.4,3.36,17.8,llano circulo figura grabada,VOGA,NaN,anillos_plata,2023-03-14,2
4,2021-11-22,AP002,T5,1.4,2.4,3.36,17.8,llano circulo figura grabada,VOGA,NaN,anillos_plata,2023-03-14,1
